In [2]:
from pyspark import SparkContext, SparkConf

**Run Hadoop locally** 
we assume hadoop is installed locally in the homedir

In [3]:
%%bash
cd ~/hadoop-2.7.3/sbin 
./stop-all.sh 
yes | hadoop namenode -format 
./hadoop-daemon.sh start namenode 
./hadoop-daemon.sh start datanode
./yarn-daemon.sh start resourcemanager
./yarn-daemon.sh start nodemanager
./mr-jobhistory-daemon.sh start historyserver
jps


This script is Deprecated. Instead use stop-dfs.sh and stop-yarn.sh
Stopping namenodes on [localhost]
localhost: ssh: connect to host localhost port 22: Connection refused
localhost: ssh: connect to host localhost port 22: Connection refused
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: ssh: connect to host 0.0.0.0 port 22: Connection refused
stopping yarn daemons
stopping resourcemanager
localhost: ssh: connect to host localhost port 22: Connection refused
no proxyserver to stop
Formatting using clusterid: CID-273ab6a0-995a-4242-ad32-2de1753caf03
namenode running as process 6193. Stop it first.
datanode running as process 6288. Stop it first.
starting resourcemanager, logging to /home/alon/hadoop-2.7.3/logs/yarn-alon-resourcemanager-alon-Latitude-E7450.out
nodemanager running as process 6456. Stop it first.
historyserver running as process 6653. Stop it first.
6288 DataNode
6193 NameNode
3508 Main
9508 Jps
9428 ResourceManager
6456 NodeManager
6653 JobHistoryServer


DEPRECATED: Use of this script to execute hdfs command is deprecated.
Instead use the hdfs command for it.

19/05/30 11:52:16 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = alon-Latitude-E7450/127.0.1.1
STARTUP_MSG:   args = [-format]
STARTUP_MSG:   version = 2.7.3
STARTUP_MSG:   classpath = /home/alon/hadoop-2.7.3/etc/hadoop:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/zookeeper-3.4.6.jar:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/snappy-java-1.0.4.1.jar:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/log4j-1.2.17.jar:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/commons-cli-1.2.jar:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/paranamer-2.3.jar:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/gson-2.2.4.jar:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/curator-recipes-2.7.1.jar:/home/alon/hadoop-2.7.3/share/hadoop/common/lib/commons-logging-1.1.3.jar:/home/alon/hadoo

**Configure Spark Session**

In [5]:
SPARK_MASTER='local'
SPARK_APP_NAME='HW2'
SPARK_EXECUTOR_MEMORY='2g'
SPARK_DRIVER_MEMORY='1g'
SPARK_EXECUTOR_CORES='3'
SPARK_CORES_MAX='4'

In [6]:
conf = SparkConf().setMaster(SPARK_MASTER) \
                      .setAppName(SPARK_APP_NAME) \
                      .set("spark.executor.memory", SPARK_EXECUTOR_MEMORY) \
                      .set('spark.executor.cores', SPARK_EXECUTOR_CORES) \
                      .set('spark.driver.memory',SPARK_DRIVER_MEMORY) \
                      .set('spark.cores.max', SPARK_CORES_MAX)

In [7]:
sc=SparkContext(conf=conf)

**It is also possible to run with the default configuration**

In [18]:
sc=SparkContext()

**Put data in HDFS**

In [14]:
# I have loaded the data from the local fs to the ROOT directory in HDFS. 
#If it is not OK you can change the destination dir for example:  hdfs dfs -put ./data /user/stud3/HW2....
!hdfs dfs -put ./data /data 

***Let's read the data from HDFS***

In [19]:
rdd=sc.textFile('hdfs:/data/microsoft-com.data').map(lambda row: row.split(','))

In [20]:
# No need for an RDD for such a small list
countries = [line.rstrip('\n') for line in open('./data/countries.txt', 'r')]

***Parse the data according to the info file. Removing redundant field and cleaning the data***

In [21]:
# Get all attribute ("A") lines with Vroots (areas of website) that match one of the countries in "countries" list
attributes=rdd.filter(lambda x: x[0]=='A') \
               .map(lambda x: ((x[0],x[1],x[3].replace('"', ''), x[4].replace('"', '')))) \
               .filter(lambda x: x[2] in countries)

In [22]:
# Get user interactions with these Vrooes
votes=rdd.filter(lambda x: x[0]=='V') \
               .map(lambda x: ((x[0],x[1],x[2])))

In [24]:
votes.take(2)

[('V', '10001', '1000'), ('V', '10001', '1001')]

In [25]:
attributes.take(2)

[('A', '1005', 'Norway', '/norge'), ('A', '1053', 'Jakarta', '/visualj')]

***Filter users that visited at least one page related to one of the given countries***

In [26]:
#Could also do it with Data Frames, but we did not study it in class so I have implemented it with RDDs
joinedRDD = votes.map(lambda x:(x[2],x[1])).join(attributes.map(lambda x:(x[1],(x[2],x[3])))) \
                 .map(lambda x:(x[1][0],  x[1][1][0], x[1][1][1]))


In [28]:
joinedRDD.take(20)

[('11035', 'Slovenija', '/slovenija'),
 ('19312', 'Slovenija', '/slovenija'),
 ('25278', 'Slovenija', '/slovenija'),
 ('25431', 'Slovenija', '/slovenija'),
 ('27083', 'Slovenija', '/slovenija'),
 ('27556', 'Slovenija', '/slovenija'),
 ('28362', 'Slovenija', '/slovenija'),
 ('33869', 'Slovenija', '/slovenija'),
 ('35728', 'Slovenija', '/slovenija'),
 ('12828', 'Finland', '/finland'),
 ('12840', 'Finland', '/finland'),
 ('13347', 'Finland', '/finland'),
 ('13837', 'Finland', '/finland'),
 ('16711', 'Finland', '/finland'),
 ('16996', 'Finland', '/finland'),
 ('18825', 'Finland', '/finland'),
 ('19327', 'Finland', '/finland'),
 ('20223', 'Finland', '/finland'),
 ('20286', 'Finland', '/finland'),
 ('20419', 'Finland', '/finland')]

***Create report: For each country, the number of users that visited a page of that country***


****Note: It is not mentioned to find the DISTINCT amount of users. each visit of a user to a country related page is counted as one, even if it is the same user****

In [29]:
z=joinedRDD.map(lambda x: (x[1],1)).reduceByKey(lambda v1,v2: v1+v2 ).sortByKey()

In [30]:
z.collect()

[('Argentina', 32),
 ('Australia', 136),
 ('Belgium', 45),
 ('Brazil', 121),
 ('Canada', 128),
 ('Caribbean', 5),
 ('Chile', 4),
 ('China', 26),
 ('Colombia', 11),
 ('Czech Republic', 16),
 ('Denmark', 55),
 ('Finland', 29),
 ('France', 183),
 ('Germany', 372),
 ('Hong Kong', 35),
 ('Hungary', 15),
 ('India', 9),
 ('Ireland', 13),
 ('Israel', 34),
 ('Italy', 167),
 ('Jakarta', 670),
 ('Korea', 94),
 ('Mexico', 33),
 ('New Zealand', 10),
 ('Norway', 42),
 ('Peru', 3),
 ('Poland', 38),
 ('Portugal', 15),
 ('Russia', 52),
 ('Slovakia', 11),
 ('Slovenija', 9),
 ('South Africa', 19),
 ('Spain', 191),
 ('Sweden', 258),
 ('Switzerland', 31),
 ('Taiwan', 204),
 ('Thailand', 11),
 ('Turkey', 9),
 ('UK', 186),
 ('Uruguay', 4),
 ('Venezuela', 8)]

***Top 5 visited Countries***

In [32]:
z.map(lambda x: (x[1],x[0])).sortByKey(ascending=False).take(5)

[(670, 'Jakarta'),
 (372, 'Germany'),
 (258, 'Sweden'),
 (204, 'Taiwan'),
 (191, 'Spain')]

***Save the first reports to HDFS and then move to local fs***

In [33]:
z.saveAsTextFile('hdfs:/data/users_per_country')

In [34]:
!hdfs dfs -getmerge /data/users_per_country /home/alon/Downloads/_results_.txt

*** Calculate Avg amount of Pages per User***

In [35]:
#Assuming we want all of the pages of the users, not just the one with countries
pagesPerUser=votes.map(lambda x: (x[1],1)).reduceByKey(lambda a,b: a+b)

In [36]:
# Calculate sum of pages and count of pages in an efficient map reduce way
sumPages, cntPages = pagesPerUser.map(lambda x: (x[1],1)).reduce(lambda a,b : (a[0]+b[0], a[1]+b[1]))

In [37]:
print('Avg pages per user is ', sumPages/cntPages)

Avg pages per user is  3.015927363883709


***Calculate Maximum pages per user***

In [38]:
# Get the user with the maximum pages, and the number of the pages he had
pagesPerUser.reduce(lambda a, b: a if a[1] > b[1] else b)

('40310', 35)

***Remove directory from HDFS***

In [39]:
!hdfs dfs -rm -r /data

19/05/30 11:40:00 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /data


In [40]:
!hdfs dfs -ls  /

Found 1 items
drwxrwx---   - alon supergroup          0 2019-05-30 11:11 /tmp
